<a href="https://colab.research.google.com/github/boitoshi/pokebros-blog/blob/main/Spreadsheet_to_Custom_HTML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Googleドライブ内のスプレッドシートをPythonで加工する
### やりたいこと


1.   Googleドライブ内のファイル（スプレッドシート）を読込
2.   データ抽出・加工
3.   カスタムHTMLのテンプレート用意
4.   書き出し



## 0.準備

1. **Google Colaboratoryの準備:**
   * ライブラリをインストール。（pip）

2. **Googleドライブへの認証:**
   * Googleドライブにアクセスするための認証
   * GoogleコラボでスプレッドシートやGoogleドライブにアクセスするには、毎回認証が必要

In [ ]:
# ライブラリインストール
!pip install google-auth-oauthlib gspread pandas

# Googleドライブの認証
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default

# GoogleCredentials.get_application_default() の代わりに default() を使用して認証情報を取得
creds, _ = default()
gc = gspread.authorize(creds)

## 1.スプレッドシートの読み込み

シークレット機能を使って読込

---
シークレットに登録方法
1. Google Colabの「編集」メニューから「ノートブックの設定」を選択します。
「シークレット」タブ→「シークレットを追加」
「名前」に任意の名前（例：SPREADSHEET_ID）、
「値」にスプレッドシートのID（スプレッドシートのURLから d/ と /edit の間）を入力
「保存」をクリック。
2. コードでシークレットを読み込む

```
from google.colab import userdata
spreadsheet_id = userdata.get('SPREADSHEET_ID')

spreadsheet = gc.open_by_key(spreadsheet_id)

# 解説:
# from google.colab import userdata で、userdata モジュールをインポート
# spreadsheet_id = userdata.get('SPREADSHEET_ID') で、シークレットとして保存したスプレッドシートIDを取得。'SPREADSHEET_ID' は、シークレットを設定した際に指定した名前。
# gc.open_by_key(spreadsheet_id) で、取得したスプレッドシートIDを使ってスプレッドシートを開く。
```

直接読み込む場合は以下のように

`spreadsheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/”スプレッドシートID”/edit#gid=0')`



In [ ]:
from google.colab import userdata
spreadsheet_id = userdata.get('SPREADSHEET_ID_ELITE4')

# gspreadを使ってスプレッドシートを開く
spreadsheet = gc.open_by_key(spreadsheet_id)

## 2.データの抽出と加工
*   スプレッドシート全体のシートを取得。
*   各シートのデータを整形して辞書形式に格納。


In [ ]:
# 必要なライブラリをインポート
import pandas as pd

# データ整形
columns = {
    'trainer': 'トレーナー', 'pokemon': 'ポケモン', 'gender': 'せいべつ', 'level': 'レベル',
    'move1': 'わざ1', 'move2': 'わざ2', 'move3': 'わざ3', 'move4': 'わざ4',
    'ability': 'とくせい', 'Nature': 'せいかく', 'IV': '個体値', 'EV': '努力値'
} # 'キー(key): 値(value) '　値(value)はスプレッドシート見出しとあわせること

# スプレッドシートのすべてのシートからデータを取得して辞書にする関数
def get_all_sheet_data(spreadsheet, columns):
    all_data = {}  # 各シートのデータをまとめる辞書

    for sheet in spreadsheet.worksheets():  # 各シートをループ
        sheet_name = sheet.title  # シート名を取得
        data = sheet.get_all_values()  # シート全体のデータ取得
        df = pd.DataFrame(data[1:], columns=data[0])  # データフレーム化（1行目をカラム名に）

        # 存在するカラムだけでデータを取得
        existing_columns = {k: v for k, v in columns.items() if v in df.columns}  # 見つかったカラムだけ辞書にする k = key（キー） v = value（値）※変数
        if not existing_columns:
            print(f"{sheet_name} シートには指定のカラムが見つからなかったよ💦")
            continue

        trainer_dict = {}  # トレーナー別のデータを保存する辞書
        current_trainer = None

        for _, row in df.iterrows():
            # トレーナー名がある場合は更新
            if row[existing_columns['trainer']] != "":
                current_trainer = row[existing_columns['trainer']]
                if current_trainer not in trainer_dict:
                    trainer_dict[current_trainer] = []

            # トレーナー名もポケモン名もない行はスキップ
            if current_trainer is None or row[existing_columns['pokemon']] == "":
                continue

            # 性別カラムがある場合だけ処理
            pokemon_name = row[existing_columns['pokemon']]
            if 'gender' in existing_columns:
                gender = row[existing_columns['gender']]
                if pd.notna(gender) and gender != "ふめい":
                    pokemon_name += f" ({gender})"

            # ポケモンデータの作成（見つかったカラムだけ）
            pokemon_data = {
                "name": pokemon_name,
                "level": row[existing_columns['level']],
                "moves": [row[existing_columns.get(f"move{i}", "")] for i in range(1, 5) if existing_columns.get(f"move{i}", "") in df.columns]
            }

            # 他の任意カラムもあれば追加
            for optional_col in ['ability', 'Nature', 'IV', 'EV']:
                if optional_col in existing_columns:
                    pokemon_data[optional_col] = row[existing_columns[optional_col]]

            # トレーナーのポケモンリストに追加
            trainer_dict[current_trainer].append(pokemon_data)

        # シートの名前をキーに辞書に保存
        all_data[sheet_name] = trainer_dict

    return all_data

 ### （確認用）シート名・トレーナー名で検索・表示
 きちんとデータが整形でいているかを事前に確認

In [ ]:
# 全シートデータの取得
all_sheet_data = get_all_sheet_data(spreadsheet, columns)

# 地方と四天王名を変更して特定のトレーナーの情報を出力する
sheet_name = 'ブルベリ'
trainer_name = 'アカマツ'

if sheet_name in all_sheet_data and trainer_name in all_sheet_data[sheet_name]:
    for pokemon in all_sheet_data[sheet_name][trainer_name]:
        print(pokemon)
else:
    print(f"{sheet_name} シートまたは {trainer_name} のデータは見つかりませんでした💦")

{'name': 'ヒートロトム', 'level': 'Lv.77', 'moves': ['おにび', 'オーバーヒート', '10まんボルト', 'たたりめ'], 'ability': 'ふゆう', 'Nature': 'ずぶとい', 'IV': '攻撃0', 'EV': 'HP244\n特攻12\n素早さ252'}
{'name': 'ファイアロー (♂)', 'level': 'Lv.77', 'moves': ['にほんばれ', 'ブレイブバード', 'フレアドライブ', 'はねやすめ'], 'ability': 'ほのおのからだ', 'Nature': 'ようき', 'IV': '全て31', 'EV': 'HP148\n攻撃108\n素早さ252'}
{'name': 'ナッシー (♂)', 'level': 'Lv.78', 'moves': ['やどりぎのタネ', 'サイコショック', 'まもる', 'ソーラービーム'], 'ability': 'しゅうかく', 'Nature': 'おだやか', 'IV': '攻撃0', 'EV': 'HP252\n防御100\n特防156'}
{'name': 'バクーダ (♂)', 'level': 'Lv.78', 'moves': ['だいちのちから', 'ねっぷう', 'げんしのちから', 'しねんのずつき'], 'ability': 'ハードロック', 'Nature': 'れいせい', 'IV': '全て31', 'EV': 'HP164\n防御108\n特防236'}
{'name': 'ブーバーン  (♂)', 'level': 'Lv.78', 'moves': ['サイコキネシス', '10まんボルト', 'ねっぷう', 'にほんばれ'], 'ability': 'ほのおのからだ', 'Nature': 'おくびょう', 'IV': '攻撃0', 'EV': 'HP164\n特攻252\n素早さ92'}
{'name': 'バシャーモ (♂)', 'level': 'Lv.79', 'moves': ['ストーンエッジ', 'きあいだま', 'やけっぱち', 'シャドークロー'], 'ability': 'かそく', 'Nature': 'いじっぱり', 'IV': '全て31', 'EV

## 3.出力するHTMLテンプレの用意
	1.	全カラム読み込み: get_trainer_data 関数内で全データを取得し、データフレームに変換。
	2.	カラムのマッピング設定: columns 辞書を使って、カラム名を指定。これにより、出力時にどのカラムを使うかを選べるようになっています。
	3.	出力時のフィルタリング: filter_trainer_data 関数を新たに追加し、指定されたカラムだけを抽出してフィルタリングする機能を実装しました。

In [ ]:
def generate_html_tables(trainer_dict):
    html_output = ""  # 最終的なHTML出力を格納する変数

    for trainer_name, pokemons in trainer_dict.items():
        # 各トレーナーのテーブルを生成
        html_output += f'<h2>{trainer_name}</h2>\n'  # トレーナー名の見出し
        html_output += '<table class="table-elite-four">\n<tbody>\n'

        # 名前の行
        name_row = '<tr>\n'
        for pokemon in pokemons[:6]:  # 最大6匹
            name_row += f'  <th><alt="{pokemon["name"]}"></th>\n'
        name_row += '</tr>\n'
        html_output += name_row

        # 性別付き名前の行
        gender_row = '<tr>\n'
        for pokemon in pokemons[:6]:
            gender_row += f'  <td align="center">{pokemon["name"]}</td>\n'
        gender_row += '</tr>\n'
        html_output += gender_row

        # レベルの行
        level_row = '<tr>\n'
        for pokemon in pokemons[:6]:
            level_row += f'  <td align="center">{pokemon["level"]}</td>\n'
        level_row += '</tr>\n'
        html_output += level_row

        # 特性の行（追加）
        if any("ability" in pokemon for pokemon in pokemons):
            ability_row = '<tr>\n'
            for pokemon in pokemons[:6]:
                ability_row += f'  <td align="center">{pokemon["ability"]}</td>\n'
            ability_row += '</tr>\n'
            html_output += ability_row

        # 技の行
        moves_row = '<tr>\n'
        for pokemon in pokemons[:6]:
            moves_row += '  <td><ul>\n'
            for move in pokemon["moves"]:
                moves_row += f'    <li>{move}</li>\n'
            moves_row += '  </ul></td>\n'
        moves_row += '</tr>\n'
        html_output += moves_row

        # 他の見出しも出したければ適宜追加する

        html_output += '</tbody>\n</table>\n'  # テーブルの閉じタグ

    return html_output

## 4.HTMLへ出力
sheet_nameとtrainer_nameで指定したトレーナーのポケモン情報を指定のHTMLに代入
出力されたものをカスタムHTMLへコピペ
※この後CSSの設定を忘れないこと

In [ ]:
# 全シートのデータを取得
all_sheet_data = get_all_sheet_data(spreadsheet, columns)

# 出力したいシートとトレーナーを指定
sheet_name = 'イッシュ'
trainer_name = 'レンブ'

# 指定したシートとトレーナー名がデータにあるか確認してからHTML出力
if sheet_name in all_sheet_data and trainer_name in all_sheet_data[sheet_name]:
    # 指定のトレーナーデータを辞書にしてHTML関数に渡す
    specific_trainer_data = {trainer_name: all_sheet_data[sheet_name][trainer_name]}
    html_tables = generate_html_tables(specific_trainer_data)
    print(html_tables)  # 指定したトレーナーのHTMLテーブルを出力
else:
    print(f"{sheet_name} シートまたは {trainer_name} のデータは見つかりませんでした💦")


<h2>レンブ</h2>
<table class="table-elite-four">
<tbody>
<tr>
  <th><alt="ナゲキ (♂)"></th>
  <th><alt="ダゲキ (♂)"></th>
  <th><alt="ローブシン (♂)"></th>
  <th><alt="コジョンド (♂)"></th>
</tr>
<tr>
  <td align="center">ナゲキ (♂)</td>
  <td align="center">ダゲキ (♂)</td>
  <td align="center">ローブシン (♂)</td>
  <td align="center">コジョンド (♂)</td>
</tr>
<tr>
  <td align="center">Lv.48</td>
  <td align="center">Lv.48</td>
  <td align="center">Lv.48</td>
  <td align="center">Lv.50</td>
</tr>
<tr>
  <td align="center">こんじょう</td>
  <td align="center">がんじょう</td>
  <td align="center">ちからずく</td>
  <td align="center">せいしんりょく</td>
</tr>
<tr>
  <td><ul>
    <li>やまあらし</li>
    <li>ストーンエッジ</li>
    <li>じならし</li>
    <li>しっぺがえし</li>
  </ul></td>
  <td><ul>
    <li>からてチョップ</li>
    <li>ストーンエッジ</li>
    <li>くさむすび</li>
    <li>かたきうち</li>
  </ul></td>
  <td><ul>
    <li>アームハンマー</li>
    <li>ストーンエッジ</li>
    <li>くさむすび</li>
    <li>かたきうち</li>
  </ul></td>
  <td><ul>
    <li>とびげり</li>
    <li>いわなだれ</li>
    <li>とんぼがえり</li>
    <li>か